In [104]:
class Coord:
    def __init__(self, x: int, y: int, value: int):
        self.x = x
        self.y = y
        self._value = value

    def is_step_to(self, other):
        return abs(self.x - other.x) + abs(self.y - other.y) == 1 and (other.value - self.value == 1)

    @property
    def value(self):
        return self._value
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __repr__(self):
        return f"({self.x},{self.y}) {self.value}"
    


class DiGraph:
    bags: dict[Coord, list[Coord]] = {}
    coords: list[Coord] = []
    def __init__(self, lines: list[list[int]]):
        self.lines = lines
        self.height = len(lines)
        self.width = len(lines[0])

        for y in range(self.height):
            for x in range(self.width):
                coord = self.coord(x, y)
                self.coords.append(coord)

                if x > 0 and coord.is_step_to(self.coord(x-1, y)):
                    self.add_edge(coord, self.coord(x-1, y))
                if x < self.width - 1 and coord.is_step_to(self.coord(x+1, y)):
                    self.add_edge(coord, self.coord(x+1, y))
                if y > 0 and coord.is_step_to(self.coord(x, y-1)):
                    self.add_edge(coord, self.coord(x, y-1))
                if y < self.height - 1 and coord.is_step_to(self.coord(x, y+1)):
                    self.add_edge(coord, self.coord(x, y+1))

    def coord(self, x: int, y: int) -> Coord:
        return Coord(x, y, self.lines[y][x])
    
    def get_trailheads(self) -> list[Coord]:
        return [coord for coord in self.coords if coord.value == 0]
    
    def get_trailends(self) -> list[Coord]:
        return [coord for coord in self.coords if coord.value == 9]
    
    def add_edge(self, a: Coord, b: Coord):
        if a not in self.bags:
            self.bags[a] = [b]
        else:
            self.bags[a].append(b)

    def get_neighbours(self, coord: Coord) -> list[Coord]:
        return self.bags.get(coord, [])
    
    def __repr__(self):
        return "\n".join("".join(str(i) for i in line) for line in self.lines)

In [105]:
import numpy as np


lines = []
with open('input.txt') as f:
    lines = [[int(i) for i in x.strip()] for x in f.readlines()]
    graph = DiGraph(lines)

graph



543067650323210321032110356789890110710189878760134567
612148941212306782345091235410765227893258759651021298
701238932301456798106787549323454336794567549842330807
898547215450589867287654678892961245987654456732498910
987656306769678956398013234781870301256003301201567821
234543215878760145478920105690210982340112210341056932
109650124981232230569834124567345673451234985452347845
018744323890341021656765033438454589298545676701037796
199803210787650110765017842129823672107655677812378987
785012345654789541874326956010510563201234389983405676
174321498723498632903455467173421454102348210234514430
065430239014323721212762398982567876218959650149623521
145540128765017890101871081071018967327968743898734678
236692134984178768718981012567894458456876212798604329
987783005673269659654108923458763349663212108789015012
896654012562154546743267830309854218767103419276126787
765780123473043432890154321212903109878756578125435698
434099874984012301740125654567812893469017647030165556
3231010657

In [106]:
class DirectedDfs:
    def __init__(self, graph: DiGraph, start: Coord):
        self.graph = graph
        self.start = start
        self.visited = set()
        self.paths: list[list[Coord]] = []
        self._dfs(start)
    
    def _dfs(self, coord: Coord):
        self.visited.add(coord)

        for neighbour in self.graph.get_neighbours(coord):
            if neighbour not in self.visited:
                self._dfs(neighbour)

    def reachable(self, coord: Coord) -> bool:
        return coord in self.visited

In [107]:
heads: list[Coord] = graph.get_trailheads()
tails: list[Coord] = graph.get_trailends()

count = 0
for head in heads:
    dfs = DirectedDfs(graph, head)
    subtotal = 0
    for tail in tails:
        if dfs.reachable(tail):
            subtotal += 1
    
    count += subtotal

print(count)

733


## Part 2

In [108]:
class DirectedDfsRank:
    paths: list[list[Coord]] = []
    def __init__(self, graph: DiGraph, start: Coord):
        self.graph = graph
        self.start = start

        self.paths.append([start])

        self.paths: list[list[Coord]] = self._dfs(start, [start])
    
    def _dfs(self, coord: Coord, path: list[Coord]) -> list[list[Coord]]:
        if coord.value == 9:
            return [path]

        neighbours = self.graph.get_neighbours(coord)
        if len(neighbours) == 0:
            return []
        
        all_paths: list[list[Coord]] = []
        for i in range(len(neighbours)):
            n = neighbours[i]

            all_paths.extend(self._dfs(n, path + [n]))
        return all_paths

    def reachable(self, coord: Coord) -> bool:
        return coord in self.visited
    
    def rank(self) -> int:
        return len(self.paths)

In [109]:
heads: list[Coord] = graph.get_trailheads()

count = 0
for head in heads:
    dfs = DirectedDfsRank(graph, head)
    rank = dfs.rank()
    print(rank)
    count += rank

print(count)

9
13
6
3
1
5
2
3
4
6
3
4
4
11
18
4
1
2
6
6
13
2
5
4
4
3
4
2
3
6
6
3
2
8
4
9
6
13
3
6
6
3
4
21
15
11
6
3
7
7
2
11
19
4
5
4
2
7
4
1
1
2
11
6
2
1
2
2
14
1
6
2
12
2
2
8
4
4
4
1
12
12
6
3
4
3
4
4
8
2
8
8
2
5
2
6
2
3
16
7
7
8
4
3
1
2
2
1
2
7
3
1
6
1
2
3
5
2
2
4
2
3
1
3
6
1
1
3
7
5
2
1
2
3
5
1
2
1
3
2
5
4
2
2
10
4
2
10
14
7
10
6
15
4
12
3
4
4
13
1
3
4
7
9
5
4
13
6
12
2
4
3
4
11
6
2
2
2
3
1
2
4
3
3
8
1
6
1
2
1
2
8
2
3
11
3
2
13
8
11
10
2
4
1
11
8
4
1
8
16
23
2
8
1
2
4
4
5
5
2
5
4
4
3
2
16
1
4
4
4
4
10
4
1
3
5
6
21
3
3
5
4
3
4
2
21
6
4
2
17
5
4
2
1
12
4
17
16
3
1
9
13
5
2
2
18
21
9
1
8
2
8
2
2
2
1
9
10
3
2
12
1514
